In [1]:
import re
import json
import chardet
from pathlib import Path
from typing import List
from autogen import Agent, AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor
import ollama
import fix_busted_json
import pandas as pd


In [2]:
# 檔案
master_js_path = r"C:\users\tuf\documents\cloudysys_nickfury\dcmsln\dcmsln_201812\Master\PageJS\Query"
js_files = list(Path(master_js_path).rglob("*.js"))

if not js_files:
    raise SystemExit("找不到 JS 檔案")

# decoder 
def read_text_file_safely(path: str) -> str:
    with open(path, "rb") as f:
        raw_data = f.read()
        detected = chardet.detect(raw_data)
        encoding = detected["encoding"] or "utf-8"

    fallback_encodings = []

    if encoding: # Add the detected encoding first
        fallback_encodings.append(encoding)
    if 'GB2312' not in fallback_encodings: # Add GB2312 if not already present
        fallback_encodings.append("GB2312")

    # Ensure common encodings are covered
    additional_encodings = ["utf-8", "big5", "cp950", "gbk", "gb18030", "utf-16", "windows-1252"]
    for enc in additional_encodings:
        if enc not in fallback_encodings:
            fallback_encodings.append(enc)

    for enc in fallback_encodings:
        try:
            decoded = raw_data.decode(enc)

            # 這裡移除常見控制字符、零寬度字元：換行、tab、回車等
            # \s 匹配所有空白字元，包括傳統空白和許多 Unicode 空白字元
            cleaned = re.sub(r'[\uFEFF\u200B\r\n\t]', '', decoded)
            
            # 去掉多餘空白並移除前後空白
            cleaned = cleaned.strip()
            return cleaned
        except UnicodeDecodeError:
            continue

    raise Exception(f"所有常見編碼皆無法解碼檔案: {path}")

# 切分 function
def extract_function_blocks(js_code: str) -> list[str]:
    blocks = []
    stack = []
    start = None
    i = 0
    while i < len(js_code):
        if js_code[i:i+8].startswith('function'):
            if not stack:
                start = i
        if js_code[i] == '{':
            stack.append(i)
        elif js_code[i] == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    blocks.append(js_code[start:i+1].strip())
                    start = None
        i += 1
    return blocks

print(f".js file count: {len(js_files)} \nfolder: {master_js_path}")


# 建立資料集
all_functions = []

for js_file in js_files:
    try:
        js_code = read_text_file_safely(str(js_file))
        functions = extract_function_blocks(js_code)
        for func in functions:
            # d = {
            #     "file": js_file.name,
            #     "function": func
            # }
            # all_functions.append(json.dumps(d, ensure_ascii=False, indent=2))
            all_functions.append({"file": js_file.name, "function": func})
    except Exception as e:
        print(f"讀取或解析失敗: {js_file}，原因: {e}")

df = pd.DataFrame(all_functions)
grouped_all_functions = df.groupby("file")["function"].apply(list).reset_index()

# 只取第一個檔案的function實驗
print(grouped_all_functions.iloc[0]["function"])


.js file count: 88 
folder: C:\users\tuf\documents\cloudysys_nickfury\dcmsln\dcmsln_201812\Master\PageJS\Query
['function InitPage(){document.all.TXT_ProdID.focus()}', 'function checkProdInfoHead(){ if((event.keyCode == 13) ||(event.keyCode == 9) ){//alert("enter checkpageinfo");var oCtl;var sLineNo;var  sResult;oCtl=document.all.TXT_ProdID;if (sTrim(oCtl.value)==""){oCtl=document.all.TXT_ProdName;oCtl.value=""event.keyCode = 9;return true;}sResult=checkProdInfo(escape(sTrim(oCtl.value)));if ( sResult.PRODID=="" ){//没有取到数据说明输入的产品代码不存在（不需要自己写报错信息）try{//清空原来的产品名稱等数据oCtl=document.all.TXT_ProdName;oCtl.value="";oCtl=document.all.TXT_ProdID;oCtl.value="";}catch(e){}event.returnValue = false;return false;      }else{//开始将取到的产品名稱等数据填到画面控件中oCtl=document.all.TXT_ProdName;oCtl.value=sResult.PRODNAMECN;oCtl=document.all.TXT_ProdID;oCtl.value=sResult.PRODID;}event.keyCode = 9;return true;  }}', 'function openProdWindow(){var result = popQuickWindow(escape(document.all.TXT_ProdID.value),POPWINDOW_P

In [3]:
# autogen setting
local_ollama_port = "http://localhost:11434"
config_list = [
    {
        "model": "qwen3:8b",
        "api_type": "ollama",
        "stream": False,
        "base_url": local_ollama_port,
    }
]

#  Agent Coding Example

In [4]:
# Setting up the code executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

# Setting up the agents

# The UserProxyAgent will execute the code that the AssistantAgent provides
user_proxy_agent = UserProxyAgent(
    name="User",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: any(
        key in msg.get("content", "") for key in ["FINISH", "Final Answer"]
    ),
    human_input_mode="NEVER"
)

system_message = """You are a helpful AI assistant who writes code and the user
executes it. Solve tasks using your javascript coding skills.
In the following cases, suggest javascript code (in a javascript coding block) for the
user to execute. When using code, you must indicate the script type in the code block.
You only need to create one working sample.
Do not suggest incomplete code which requires users to modify it.
Don't use a code block if it's not intended to be executed by the user. Don't
include multiple code blocks in one response. Do not ask users to copy and
paste the result. Instead, use 'print' function for the output when relevant.
Check the execution result returned by the user.

If the result indicates there is an error, fix the error.

IMPORTANT: If it has executed successfully, ONLY output 'FINISH'."""

# The AssistantAgent, using the Ollama config, will take the coding request and return code
assistant_agent = AssistantAgent(
    name="Ollama Assistant",
    system_message=system_message,
    llm_config={"config_list": config_list},
)

In [5]:
# Start the chat, with the UserProxyAgent asking the AssistantAgent the message
chat_result = user_proxy_agent.initiate_chat(
    assistant_agent,
    message=f"Provide code to rewrite or improve the function: {grouped_all_functions.iloc[0]['function'][0]}",
)

User (to Ollama Assistant):

Provide code to rewrite or improve the function: function InitPage(){document.all.TXT_ProdID.focus()}

--------------------------------------------------------------------------------
Ollama Assistant (to User):

<think>
Okay, let's see. The user provided a function InitPage that focuses on an element with the ID TXT_ProdID using document.all. I remember that document.all is an older way of selecting elements, and it's not recommended for modern JavaScript. So the goal is to rewrite or improve this function.

First, I should check if the element exists. Using document.all might not work in some contexts, especially if the element is dynamically loaded or if the script runs before the element is available. So maybe using document.getElementById is better because it's the standard method.

Wait, the original code uses document.all, which is specific to Internet Explorer. So the improved version should use a more compatible approach. Also, adding a check to se

# Three Agents Coding Example

In [6]:
model_names = ["qwen3:8b", "gemma3:4b", "deepcoder:1.5b"]

In [12]:
# Shared system message
system_message = """You are a helpful AI assistant who writes code and the user
executes it. Solve tasks using your javascript coding skills.
In the following cases, suggest javascript code (in a javascript coding block) for the
user to execute. When using code, you must indicate the script type in the code block.
You only need to create one working sample.
Do not suggest incomplete code which requires users to modify it.
Don't use a code block if it's not intended to be executed by the user. Don't
include multiple code blocks in one response. Do not ask users to copy and
paste the result. Instead, use 'print' function for the output when relevant.
Check the execution result returned by the user.

If the result indicates there is an error, fix the error.

IMPORTANT: If it has executed successfully, ONLY output 'FINISH'."""

In [13]:
# Create working directory and executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

In [14]:
# Create UserProxyAgent
user_proxy_agent = UserProxyAgent(
    name="User",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: any(
        key in msg.get("content", "") for key in ["FINISH", "Final Answer"]
    ),
    human_input_mode="NEVER"
)

In [15]:
# Create AssistantAgents for each model
assistant_agents = []
for model in model_names:
   agent = AssistantAgent(
       name=f"Assistant_{model.replace(':', '_')}",
       system_message=system_message,
       llm_config={
           "config_list": [
               {
                   "model": model,
                   "api_type": "ollama",
                   "stream": False,
                   "base_url": local_ollama_port,
               }
            ]
        },
   )
   assistant_agents.append(agent)

In [16]:
# Run the task with each assistant in sequence
task = f"Provide code to rewrite or improve the function. {grouped_all_functions.iloc[0]['function'][0]}"

for assistant in assistant_agents:
    print(f"\n=== Running with {assistant.name} ===")
    result = user_proxy_agent.initiate_chat(
        assistant,
        message=task
    )


=== Running with Assistant_qwen3_8b ===
User (to Assistant_qwen3_8b):

Provide code to rewrite or improve the function. function InitPage(){document.all.TXT_ProdID.focus()}

--------------------------------------------------------------------------------
Assistant_qwen3_8b (to User):

<think>
Okay, let's see. The user provided a function InitPage() that focuses on an element with the ID TXT_ProdID using document.all. I need to rewrite or improve this function.

First, I remember that document.all is an older way of selecting elements, primarily used in Internet Explorer. Modern browsers don't support it, so using document.getElementById is better. Also, the function is supposed to focus on the element, so I should make sure the element exists before trying to focus.

Wait, the original code uses document.all, which might not work in current browsers. So the improved version should use the standard method. Let me check: in modern JavaScript, we use document.getElementById or querySelec